
<h1 id="Standard-Pooling-Problem">Standard Pooling Problem<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Standard-Pooling-Problem">¶</a></h1><h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Objective-and-Prerequisites">¶</a></h2><p>One of the new features of Gurobi 9.0 is the addition of a bilinear solver, which enables finding the optimal solution of non-convex quadratic programming  problems (i.e. QPs, QCQPs, MIQPs, and MIQCQPs). This notebook will show you how to use this feature by tackling an instance of the Standard Pooling Problem, one of the most well-known Bilinear Programs. Moreover, we will present two alternative formulations, casted as Quadratically Constrained Quadratic Problems, to contrast their performance when calling the optimizer.</p>
<p>This modeling example is at the advanced level. To fully understand the content of this notebook, the reader should be familiar with the following:</p>
<ul>
<li>Python.</li>
<li>Gurobi Python interface.</li>
<li>Advanced knowledge of building mathematical optimization models. Typically, the constraints of these Jupyter Notebooks are complex or require advanced features of the Gurobi Python Interface.</li>
</ul>
<p><strong>Download the Repository</strong> <br/> 
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/>
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-OGS-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_standard-pooling_COM_EVAL_GITHUB_&amp;utm_term=standard-pooling-problem&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-OGS-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_standard-pooling_ACADEMIC_EVAL_GITHUB_&amp;utm_term=standard-pooling-problem&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>
<hr/>
<h2 id="Motivation">Motivation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Motivation">¶</a></h2><p>The pooling problem is a challenging problem in the petrochemical refining, wastewater treatment and mining industries. This problem can be regarded as a generalization of the minimum-cost flow problem and the blending problem. It is indeed important because of the significant savings it can generate, so it comes at no surprise that it has been studied extensively since Haverly pointed out the non-linear structure of this problem in 1978 [5].</p>
<hr/>
<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Problem-Description">¶</a></h2><p>The Minimum-Cost Flow Problem (MCFP) seeks to find the cheapest way of sending a certain amount of flow from a set of source nodes to a set of target nodes, possibily via transshipment nodes⁠, in a directed capacitated network. The Blending Problem is a type of MCFP with only source and target nodes, where raw materials with different attribute qualities are blended together to create end products in such a way that their attribute qualities are within tolerances.</p>
<p>The Pooling Problem combines features of both problems, as flow streams from different sources are mixed at intermediate pools and blended again at the target nodes. The non-linearity is in fact the direct result of considering pools, as the quality of a given attribute at a pool —defined as the weighted average of the qualities of the incoming streams— is an unknown quantity and thus needs to be captured by a decision variable. We refer to this problem as the Standard Pooling Problem when the network can be represented by a tripartite graph, i.e. three disjoint sets of nodes such that no nodes within the same set are adjacent. In a nutshell, it can be stated as follows: Given a list of source nodes with raw materials containing known attribute qualities, what is the cheapest way of mixing these materials at intermediate pools so as to meet the demand and tolerances at multiple target nodes? (Gupte et al., 2017) [4]. Several different formulations for the Standard Pooling Problem and its extensions exist in the literature, which can be classified into two main categories: one that consists of flow and quality variables, and the other that uses flow proportions instead of quality variables. Both categories will be considered in this notebook.</p>
<hr/>
<h2 id="Problem-Instance">Problem Instance<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Problem-Instance">¶</a></h2><p>As an illustrative example, we will solve the second Pooling Problem posed by Rehfeldt and Tisljar in 1997 and cited by Audet et al. in 2004:</p>
<p><img alt="Graph" src="./std_pooling_files/rehfeldt_tisljar2_graph.png"/></p>
<p><img alt="Table" src="./std_pooling_files/rehfeldt_tisljar2_table.png"/></p>
<p>To that end, let's declare the required data structures to represent this problem instance:</p>


In [ ]:

import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

attrs = {'den', 'bnz', 'roz', 'moz'}

sources, cost, supply, content = gp.multidict({
    "s1": [49.2, 6097.56, {'den': 0.82, 'bnz':3, 'roz':99.2,'moz':90.5}],
    "s2": [62.0, 16129, {'den': 0.62, 'bnz':0, 'roz':87.9,'moz':83.5}],
    "s3": [300.0, 500, {'den': 0.75, 'bnz':0, 'roz':114,'moz':98.7}]
})

targets, price, demand, min_tol, max_tol = gp.multidict({
    "t1": [190, 500, {'den': 0.74, 'roz':95,'moz':85}, {'den': 0.79}],
    "t2": [230, 500, {'den': 0.74, 'roz':96,'moz':88}, {'den': 0.79, 'bnz':0.9}],
    "t3": [150, 500, {'den': 0.74, 'roz':91}, {'den': 0.79}]
})

pools, cap = gp.multidict({
    "p1": 1250,
    "p2": 1750
})

# The function `product` deploys the Cartesian product of elements in sets A and B
s2p = set(product(sources, pools))
p2t = set(product(pools, targets))
s2t = {("s1", "t2"),
       ("s2", "t1"),
       ("s2", "t3"),
       ("s3", "t1")}




<hr/>
<h2 id="Solution-Approach">Solution Approach<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Solution-Approach">¶</a></h2><p>Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.</p>
<p>A mathematical optimization model has five components, namely:</p>
<ul>
<li>Sets and indices.</li>
<li>Parameters.</li>
<li>Decision variables.</li>
<li>Objective function(s).</li>
<li>Constraints.</li>
</ul>
<p>A quadratic constraint that involves only products of disjoint pairs of variables is often called a bilinear constraint, and a model that contains bilinear constraints is often called a Bilinear Program. Bilinear constraints are a special case of non-convex quadratic constraints. This type of problems is typically solved using spatial Branch and Bound (sB&amp;B). This algorithm explores the entire search space, so it provides a globally valid lower bound on the optimal objective value and —given enough time— it will find a globally optimal solution (subject to tolerances). The interested reader is referred to <a href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#references">references</a> [3], [6] and [7].</p>
<p>We now present two alternative Bilinear Programs for the Standard Pooling Problem:</p>
<h3 id="P-formulation-(Concentration)">P-formulation (Concentration)<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#P-formulation-(Concentration)">¶</a></h3><h4 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Sets-and-Indices">¶</a></h4><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;G&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;E&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 5.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.884em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.76em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Math-italic;">G</span><span class="mo" id="MathJax-Span-4" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-5" style="font-family: MathJax_Main; padding-left: 0.301em;">(</span><span class="mi" id="MathJax-Span-6" style="font-family: MathJax_Math-italic;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span class="mo" id="MathJax-Span-7" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-8" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">E<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-9" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>G</mi><mo>=</mo><mo stretchy="false">(</mo><mi>V</mi><mo>,</mo><mi>E</mi><mo stretchy="false">)</mo></math></span></span><script id="MathJax-Element-1" type="math/tex">G=(V,E)</script>: Directed graph.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-10" style="width: 3.932em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.277em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1003.28em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-11"><span class="mi" id="MathJax-Span-12" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-13" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-14" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-15" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-16" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi><mo>,</mo><mi>j</mi><mo>∈</mo><mi>V</mi></math></span></span><script id="MathJax-Element-2" type="math/tex">i,j \in V</script>: Set of nodes.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;E&lt;/mi&gt;&lt;mo&gt;&amp;#x2282;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-17" style="width: 9.824em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.158em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1008.16em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-18"><span class="mo" id="MathJax-Span-19" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-20" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-21" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-22" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-23" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-24" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-25" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">E<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-26" style="font-family: MathJax_Main; padding-left: 0.301em;">⊂</span><span class="mi" id="MathJax-Span-27" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span class="mo" id="MathJax-Span-28" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mi" id="MathJax-Span-29" style="font-family: MathJax_Math-italic; padding-left: 0.241em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo stretchy="false">(</mo><mi>i</mi><mo>,</mo><mi>j</mi><mo stretchy="false">)</mo><mo>∈</mo><mi>E</mi><mo>⊂</mo><mi>V</mi><mo>×</mo><mi>V</mi></math></span></span><script id="MathJax-Element-3" type="math/tex">(i,j) \in E \subset  V \times V</script>: Set of edges.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;mo&gt;&amp;#x2223;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;E&lt;/mi&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-30" style="width: 14.824em; display: inline-block;"><span style="display: inline-block; position: relative; width: 12.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1012.26em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-31"><span class="mi" id="MathJax-Span-32" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-33" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-34" style="font-family: MathJax_Math-italic;">i</span><span class="msubsup" id="MathJax-Span-35"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1000.3em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-36" style="font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.42em;"><span class="mo" id="MathJax-Span-37" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-38" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-39" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mi" id="MathJax-Span-40" style="font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-41" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-42" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span class="mo" id="MathJax-Span-43" style="font-family: MathJax_Main; padding-left: 0.301em;">∣</span><span class="mo" id="MathJax-Span-44" style="font-family: MathJax_Main; padding-left: 0.301em;">(</span><span class="mi" id="MathJax-Span-45" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-46" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-47" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-48" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-49" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-50" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">E<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-51" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>N</mi><mo stretchy="false">(</mo><mi>i</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup><mo>=</mo><mo fence="false" stretchy="false">{</mo><mi>j</mi><mo>∈</mo><mi>V</mi><mo>∣</mo><mo stretchy="false">(</mo><mi>i</mi><mo>,</mo><mi>j</mi><mo stretchy="false">)</mo><mo>∈</mo><mi>E</mi><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-4" type="math/tex">N(i)^+ = \{j \in V \mid (i,j) \in E \}</script>: Set of successor nodes receiving outflow from node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-52" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-53"><span class="mi" id="MathJax-Span-54" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-5" type="math/tex">i</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;mo&gt;&amp;#x2223;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;E&lt;/mi&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-55" style="width: 14.824em; display: inline-block;"><span style="display: inline-block; position: relative; width: 12.324em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1012.26em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-56"><span class="mi" id="MathJax-Span-57" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-58" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-59" style="font-family: MathJax_Math-italic;">j</span><span class="msubsup" id="MathJax-Span-60"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1000.3em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-61" style="font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.42em;"><span class="mo" id="MathJax-Span-62" style="font-size: 70.7%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-63" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-64" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mi" id="MathJax-Span-65" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-66" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-67" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span class="mo" id="MathJax-Span-68" style="font-family: MathJax_Main; padding-left: 0.301em;">∣</span><span class="mo" id="MathJax-Span-69" style="font-family: MathJax_Main; padding-left: 0.301em;">(</span><span class="mi" id="MathJax-Span-70" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-71" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-72" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-73" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-74" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-75" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">E<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-76" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>N</mi><mo stretchy="false">(</mo><mi>j</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>=</mo><mo fence="false" stretchy="false">{</mo><mi>i</mi><mo>∈</mo><mi>V</mi><mo>∣</mo><mo stretchy="false">(</mo><mi>i</mi><mo>,</mo><mi>j</mi><mo stretchy="false">)</mo><mo>∈</mo><mi>E</mi><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-6" type="math/tex">N(j)^- = \{i \in V \mid (i,j) \in E \}</script>: Set of predecessor nodes sending inflow to node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-77" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-78"><span class="mi" id="MathJax-Span-79" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-7" type="math/tex">i</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Attrs&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-80" style="width: 4.943em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.11em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1004.05em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-81"><span class="mi" id="MathJax-Span-82" style="font-family: MathJax_Math-italic;">k</span><span class="mo" id="MathJax-Span-83" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-84" style="font-family: MathJax_Main; padding-left: 0.301em;">Attrs</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.075em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi><mo>∈</mo><mtext>Attrs</mtext></math></span></span><script id="MathJax-Element-8" type="math/tex">k \in \text{Attrs}</script>: Set of attributes.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;mo&gt;&amp;#x2282;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-85" style="width: 8.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.146em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1007.15em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-86"><span class="mi" id="MathJax-Span-87" style="font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-88" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-89" style="font-family: MathJax_Main; padding-left: 0.301em;">Sources</span><span class="mo" id="MathJax-Span-90" style="font-family: MathJax_Main; padding-left: 0.301em;">⊂</span><span class="mi" id="MathJax-Span-91" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.075em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><mo>∈</mo><mtext>Sources</mtext><mo>⊂</mo><mi>V</mi></math></span></span><script id="MathJax-Element-9" type="math/tex">s \in \text{Sources} \subset V</script>: Set of source nodes, i.e. <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2205;&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-92" style="width: 5.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.646em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.59em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-93"><span class="mi" id="MathJax-Span-94" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-95" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-96" style="font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-97"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1000.3em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-98" style="font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.42em;"><span class="mo" id="MathJax-Span-99" style="font-size: 70.7%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-100" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mi" id="MathJax-Span-101" style="font-family: MathJax_Main; padding-left: 0.301em;">∅</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>=</mo><mi mathvariant="normal">∅</mi></math></span></span><script id="MathJax-Element-10" type="math/tex">N(s)^-= \emptyset</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;mo&gt;&amp;#x2282;&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-102" style="width: 8.515em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.086em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1007.09em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-103"><span class="mi" id="MathJax-Span-104" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-105" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-106" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span><span class="mo" id="MathJax-Span-107" style="font-family: MathJax_Main; padding-left: 0.301em;">⊂</span><span class="mi" id="MathJax-Span-108" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi><mo>∈</mo><mtext>Targets</mtext><mo>⊂</mo><mi>V</mi></math></span></span><script id="MathJax-Element-11" type="math/tex">t \in \text{Targets} \subset V</script>: Set of target nodes, i.e. <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2205;&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-109" style="width: 5.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.47em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-110"><span class="mi" id="MathJax-Span-111" style="font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-112" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-113" style="font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-114"><span style="display: inline-block; position: relative; width: 1.015em; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1000.3em, 4.408em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-115" style="font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.42em;"><span class="mo" id="MathJax-Span-116" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-117" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mi" id="MathJax-Span-118" style="font-family: MathJax_Main; padding-left: 0.301em;">∅</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup><mo>=</mo><mi mathvariant="normal">∅</mi></math></span></span><script id="MathJax-Element-12" type="math/tex">N(t)^+= \emptyset</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mi&gt;V&lt;/mi&gt;&lt;mo class="MJX-variant"&gt;&amp;#x2216;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;mo&gt;&amp;#x222A;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-119" style="width: 18.932em; display: inline-block;"><span style="display: inline-block; position: relative; width: 15.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1015.66em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-120"><span class="mi" id="MathJax-Span-121" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-122" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-123" style="font-family: MathJax_Main; padding-left: 0.301em;">Pools</span><span class="mo" id="MathJax-Span-124" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mi" id="MathJax-Span-125" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">V<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.182em;"></span></span><span class="mo" id="MathJax-Span-126" style="font-family: MathJax_Main; padding-left: 0.241em;">∖</span><span class="mo" id="MathJax-Span-127" style="font-family: MathJax_Main; padding-left: 0.241em;">(</span><span class="mtext" id="MathJax-Span-128" style="font-family: MathJax_Main;">Sources</span><span class="mo" id="MathJax-Span-129" style="font-family: MathJax_Main; padding-left: 0.241em;">∪</span><span class="mtext" id="MathJax-Span-130" style="font-family: MathJax_Main; padding-left: 0.241em;">Targets</span><span class="mo" id="MathJax-Span-131" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi><mo>∈</mo><mtext>Pools</mtext><mo>=</mo><mi>V</mi><mo class="MJX-variant">∖</mo><mo stretchy="false">(</mo><mtext>Sources</mtext><mo>∪</mo><mtext>Targets</mtext><mo stretchy="false">)</mo></math></span></span><script id="MathJax-Element-13" type="math/tex">p \in \text{Pools} = V \setminus (\text{Sources} \cup \text{Targets})</script>: Set of pools.</p>
<h4 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Parameters">¶</a></h4><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Cost&lt;/mtext&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-132" style="width: 6.074em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.063em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.06em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-133"><span class="msubsup" id="MathJax-Span-134"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-135" style="font-family: MathJax_Main;">Cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.027em;"><span class="mi" id="MathJax-Span-136" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-137" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-138" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-139"><span class="mrow" id="MathJax-Span-140"><span class="mi" id="MathJax-Span-141" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-142" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Cost</mtext><mi>s</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-14" type="math/tex">\text{Cost}_s \in \mathbb{R}^+</script>: Cost of acquiring one unit of raw material at source node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-143" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-144"><span class="mi" id="MathJax-Span-145" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-15" type="math/tex">s</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Supply&lt;/mtext&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-146" style="width: 7.324em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.074em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.07em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-147"><span class="msubsup" id="MathJax-Span-148"><span style="display: inline-block; position: relative; width: 3.455em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-149" style="font-family: MathJax_Main;">Supply</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="mi" id="MathJax-Span-150" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-151" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-152" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-153"><span class="mrow" id="MathJax-Span-154"><span class="mi" id="MathJax-Span-155" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-156" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Supply</mtext><mi>s</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-16" type="math/tex">\text{Supply}_s \in \mathbb{R}^+</script>: Maximum number of units of raw material available at source node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-157" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-158"><span class="mi" id="MathJax-Span-159" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-17" type="math/tex">s</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-160" style="width: 8.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.146em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.15em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-161"><span class="msubsup" id="MathJax-Span-162"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.51em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-163" style="font-family: MathJax_Main;">Content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.574em;"><span class="texatom" id="MathJax-Span-164"><span class="mrow" id="MathJax-Span-165"><span class="mi" id="MathJax-Span-166" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-167" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-168" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-169" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-170" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-171"><span class="mrow" id="MathJax-Span-172"><span class="mi" id="MathJax-Span-173" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-174" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-18" type="math/tex">\text{Content}_{s,k} \in \mathbb{R}^+</script>: Content of attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-175" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-176"><span class="mi" id="MathJax-Span-177" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-19" type="math/tex">k</script> in raw material at source node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-178" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-179"><span class="mi" id="MathJax-Span-180" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-20" type="math/tex">s</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Price&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-181" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.18em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-182"><span class="msubsup" id="MathJax-Span-183"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.21em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-184" style="font-family: MathJax_Main;">Price</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.265em;"><span class="mi" id="MathJax-Span-185" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-186" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-187" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-188"><span class="mrow" id="MathJax-Span-189"><span class="mi" id="MathJax-Span-190" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-191" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Price</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-21" type="math/tex">\text{Price}_t \in \mathbb{R}^+</script>: Price for selling one unit of final blend at target node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-192" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-193"><span class="mi" id="MathJax-Span-194" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-22" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Demand&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-195" style="width: 7.979em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.61em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.61em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-196"><span class="msubsup" id="MathJax-Span-197"><span style="display: inline-block; position: relative; width: 3.991em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.63em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-198" style="font-family: MathJax_Main;">Demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.634em;"><span class="mi" id="MathJax-Span-199" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-200" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-201" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-202"><span class="mrow" id="MathJax-Span-203"><span class="mi" id="MathJax-Span-204" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-205" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Demand</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-23" type="math/tex">\text{Demand}_t \in \mathbb{R}^+</script>: Minimum number of units required of final blend at target node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-206" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-207"><span class="mi" id="MathJax-Span-208" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-24" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Min_tol&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-209" style="width: 8.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.908em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.91em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-210"><span class="msubsup" id="MathJax-Span-211"><span style="display: inline-block; position: relative; width: 4.289em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.4em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-212" style="font-family: MathJax_Main;">Min_tol</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.396em;"><span class="texatom" id="MathJax-Span-213"><span class="mrow" id="MathJax-Span-214"><span class="mi" id="MathJax-Span-215" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-216" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-217" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-218" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-219" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-220"><span class="mrow" id="MathJax-Span-221"><span class="mi" id="MathJax-Span-222" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-223" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Min_tol</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-25" type="math/tex">\text{Min_tol}_{t,k} \in \mathbb{R}^+</script>: Minimum tolerance for attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-224" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-225"><span class="mi" id="MathJax-Span-226" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-26" type="math/tex">k</script> in final blend at target node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-227" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-228"><span class="mi" id="MathJax-Span-229" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-27" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Max_tol&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-230" style="width: 8.574em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.146em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.15em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-231"><span class="msubsup" id="MathJax-Span-232"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.57em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-233" style="font-family: MathJax_Main;">Max_tol</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.634em;"><span class="texatom" id="MathJax-Span-234"><span class="mrow" id="MathJax-Span-235"><span class="mi" id="MathJax-Span-236" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-237" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-238" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-239" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-240" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-241"><span class="mrow" id="MathJax-Span-242"><span class="mi" id="MathJax-Span-243" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-244" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Max_tol</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-28" type="math/tex">\text{Max_tol}_{t,k} \in \mathbb{R}^+</script>: Maximum tolerance for attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-245" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-246"><span class="mi" id="MathJax-Span-247" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-29" type="math/tex">k</script> in final blend at target node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-248" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-249"><span class="mi" id="MathJax-Span-250" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-30" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;Cap&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-31-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-251" style="width: 5.836em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1004.82em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-252"><span class="msubsup" id="MathJax-Span-253"><span style="display: inline-block; position: relative; width: 2.205em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.73em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-254" style="font-family: MathJax_Main;">Cap</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 1.789em;"><span class="mi" id="MathJax-Span-255" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-256" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-257" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-258"><span class="mrow" id="MathJax-Span-259"><span class="mi" id="MathJax-Span-260" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-261" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>Cap</mtext><mi>p</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-31" type="math/tex">\text{Cap}_p \in \mathbb{R}^+</script>: Maximum Capacity to store intermediate blend at pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-32-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-262" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-263"><span class="mi" id="MathJax-Span-264" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-32" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;UB&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-33-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-265" style="width: 5.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.884em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1004.88em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-266"><span class="msubsup" id="MathJax-Span-267"><span style="display: inline-block; position: relative; width: 2.265em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-268" style="font-family: MathJax_Main;">UB</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.432em;"><span class="texatom" id="MathJax-Span-269"><span class="mrow" id="MathJax-Span-270"><span class="mi" id="MathJax-Span-271" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-272" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-273" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-274" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-275" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-276"><span class="mrow" id="MathJax-Span-277"><span class="mi" id="MathJax-Span-278" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-279" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>UB</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-33" type="math/tex">\text{UB}_{i,j}\in \mathbb{R}^+</script>: Maximum flow from node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-34-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-280" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-281"><span class="mi" id="MathJax-Span-282" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-34" type="math/tex">i</script> to node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-35-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-283" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-284"><span class="mi" id="MathJax-Span-285" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-35" type="math/tex">j</script>.</p>
<h4 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Decision-Variables">¶</a></h4><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;UB&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-36-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-286" style="width: 9.229em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1007.56em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-287"><span class="msubsup" id="MathJax-Span-288"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-289" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-290"><span class="mrow" id="MathJax-Span-291"><span class="mi" id="MathJax-Span-292" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-293" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-294" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-295" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-296" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-297" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-298" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-299" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 2.265em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-300" style="font-family: MathJax_Main;">UB</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.432em;"><span class="texatom" id="MathJax-Span-301"><span class="mrow" id="MathJax-Span-302"><span class="mi" id="MathJax-Span-303" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-304" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-305" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-306" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><msub><mtext>UB</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-36" type="math/tex">\text{flow}_{i,j} \in [0, \text{UB}_{i,j}]</script>: Flow from node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-37-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-307" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-308"><span class="mi" id="MathJax-Span-309" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-37" type="math/tex">i</script> to node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-38-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-310" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-311"><span class="mi" id="MathJax-Span-312" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-38" type="math/tex">j</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;quality&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-39-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-313" style="width: 8.039em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.67em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.67em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-314"><span class="msubsup" id="MathJax-Span-315"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-316" style="font-family: MathJax_Main;">quality</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="texatom" id="MathJax-Span-317"><span class="mrow" id="MathJax-Span-318"><span class="mi" id="MathJax-Span-319" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-320" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-321" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-322" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-323" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-324"><span class="mrow" id="MathJax-Span-325"><span class="mi" id="MathJax-Span-326" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-327" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>quality</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-39" type="math/tex">\text{quality}_{p,k} \in \mathbb{R}^+</script>: Concentration of attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-40-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-328" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-329"><span class="mi" id="MathJax-Span-330" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-40" type="math/tex">k</script> at pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-41-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-331" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-332"><span class="mi" id="MathJax-Span-333" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-41" type="math/tex">p</script>.</p>
<h4 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Objective-Function">¶</a></h4><ul>
<li><strong>Profit</strong>: Maximize total profits.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Price&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Cost&lt;/mtext&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-42-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-334" style="width: 100%; display: inline-block; min-width: 34.467em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 34.467em;"><span style="position: absolute; clip: rect(2.324em, 1030.36em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-335"><span class="mtable" id="MathJax-Span-336" style="min-width: 34.467em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 34.467em;"><span style="display: inline-block; position: absolute; width: 30.36em; height: 0px; clip: rect(-1.664em, 1030.36em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -15.176em;"><span style="position: absolute; clip: rect(2.324em, 1030.36em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 30.36em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1030.36em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -15.176em;"><span class="mtd" id="MathJax-Span-340"><span class="mrow" id="MathJax-Span-341"><span class="mtext" id="MathJax-Span-342" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-343" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-344" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-345" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-346" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.098em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.836em;"><span class="mo" id="MathJax-Span-347" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.04em, 4.408em, -999.997em); top: -2.914em; left: 0em;"><span class="texatom" id="MathJax-Span-348"><span class="mrow" id="MathJax-Span-349"><span class="mi" id="MathJax-Span-350" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-351" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-352" style="font-size: 70.7%; font-family: MathJax_Main;">Targets</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-353" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-354"><span class="munderover" id="MathJax-Span-355"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-356" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-357"><span class="mrow" id="MathJax-Span-358"><span class="mi" id="MathJax-Span-359" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-360" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-361" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-362" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-363" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-364"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-365" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-366" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-367" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-368"><span class="msubsup" id="MathJax-Span-369"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.21em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-370" style="font-family: MathJax_Main;">Price</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.265em;"><span class="mi" id="MathJax-Span-371" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-372" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-373" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-374" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-375"><span class="mrow" id="MathJax-Span-376"><span class="mi" id="MathJax-Span-377" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-378" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-379" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span><span class="mo" id="MathJax-Span-380" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-381" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.158em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.836em;"><span class="mo" id="MathJax-Span-382" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.1em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-383"><span class="mrow" id="MathJax-Span-384"><span class="mi" id="MathJax-Span-385" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-386" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-387" style="font-size: 70.7%; font-family: MathJax_Main;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-388" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-389"><span class="munderover" id="MathJax-Span-390"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-391" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.74em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-392"><span class="mrow" id="MathJax-Span-393"><span class="mi" id="MathJax-Span-394" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-395" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-396" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-397" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-398" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-399"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-400" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-401" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-402" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-403"><span class="msubsup" id="MathJax-Span-404"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.97em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-405" style="font-family: MathJax_Main;">Cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.027em;"><span class="mi" id="MathJax-Span-406" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-407" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-408" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-409" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-410"><span class="mrow" id="MathJax-Span-411"><span class="mi" id="MathJax-Span-412" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-413" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-414" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-0"><span class="mrow" id="MathJax-Span-338"><span class="mtext" id="MathJax-Span-339" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Targets</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Price</mtext><mi>t</mi></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow></mrow><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mtext>Sources</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Cost</mtext><mi>s</mi></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>j</mi></mrow></msub></mrow></mrow></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-42" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{t \in \text{Targets}}{\sum_{i \in N(t)^-}{\text{Price}_t \cdot \text{flow}_{i,t}}} - \sum_{s \in \text{Sources}}{\sum_{j \in N(s)^+}{\text{Cost}_s \cdot \text{flow}_{s,j}}}
\tag{0}
\end{equation}</script><h4 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Constraints">¶</a></h4><ul>
<li><strong>Flow conservation</strong>: Total inflow of pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-43-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-415" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-416"><span class="mi" id="MathJax-Span-417" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-43" type="math/tex">p</script> must be equal to its total outflow (nothing is stored in them).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-44-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-418" style="width: 100%; display: inline-block; min-width: 24.289em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 24.289em;"><span style="position: absolute; clip: rect(2.324em, 1020.06em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-419"><span class="mtable" id="MathJax-Span-420" style="min-width: 24.289em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 24.289em;"><span style="display: inline-block; position: absolute; width: 20.122em; height: 0px; clip: rect(-1.664em, 1020.06em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -10.057em;"><span style="position: absolute; clip: rect(2.324em, 1020.06em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 20.122em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1020.06em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -10.057em;"><span class="mtd" id="MathJax-Span-424"><span class="mrow" id="MathJax-Span-425"><span class="munderover" id="MathJax-Span-426"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-427" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-428"><span class="mrow" id="MathJax-Span-429"><span class="mi" id="MathJax-Span-430" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-431" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-432" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-433" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-434" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-435"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-436" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-437" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-438" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-439"><span class="msubsup" id="MathJax-Span-440"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-441" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-442"><span class="mrow" id="MathJax-Span-443"><span class="mi" id="MathJax-Span-444" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-445" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-446" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-447" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-448" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-449" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.8em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-450"><span class="mrow" id="MathJax-Span-451"><span class="mi" id="MathJax-Span-452" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-453" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-454" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-455" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-456" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-457"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-458" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-459" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-460" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-461"><span class="msubsup" id="MathJax-Span-462"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-463" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-464"><span class="mrow" id="MathJax-Span-465"><span class="mi" id="MathJax-Span-466" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-467" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-468" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-469" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-470" style="font-family: MathJax_Main; padding-left: 0.301em;">0</span><span class="mspace" id="MathJax-Span-471" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-472" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-473" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-474" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-475" style="font-family: MathJax_Main; padding-left: 0.301em;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-1"><span class="mrow" id="MathJax-Span-422"><span class="mtext" id="MathJax-Span-423" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>=</mo><mn>0</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Pools</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-44" type="math/tex; mode=display">\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} - \sum_{s \in N(p)^-}{\text{flow}_{s,p}} = 0 \quad \forall p \in \text{Pools}
\tag{1}
\end{equation}</script><ul>
<li><strong>Source capacity</strong>: Total outflow of source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-45-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-476" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-477"><span class="mi" id="MathJax-Span-478" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-45" type="math/tex">s</script> cannot exceed its capacity.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Supply&lt;/mtext&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-46-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-479" style="width: 100%; display: inline-block; min-width: 21.134em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 21.134em;"><span style="position: absolute; clip: rect(2.324em, 1016.97em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-480"><span class="mtable" id="MathJax-Span-481" style="min-width: 21.134em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 21.134em;"><span style="display: inline-block; position: absolute; width: 16.967em; height: 0px; clip: rect(-1.664em, 1016.97em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -8.509em;"><span style="position: absolute; clip: rect(2.324em, 1016.97em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 16.967em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1016.97em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -8.509em;"><span class="mtd" id="MathJax-Span-485"><span class="mrow" id="MathJax-Span-486"><span class="munderover" id="MathJax-Span-487"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-488" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.74em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-489"><span class="mrow" id="MathJax-Span-490"><span class="mi" id="MathJax-Span-491" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span><span class="mo" id="MathJax-Span-492" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-493" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-494" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-495" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-496"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-497" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-498" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-499" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-500"><span class="msubsup" id="MathJax-Span-501"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-502" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-503"><span class="mrow" id="MathJax-Span-504"><span class="mi" id="MathJax-Span-505" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-506" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-507" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-508" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-509" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.455em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-510" style="font-family: MathJax_Main;">Supply</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="mi" id="MathJax-Span-511" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-512" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-513" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-514" style="font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-515" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-516" style="font-family: MathJax_Main; padding-left: 0.301em;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-2"><span class="mrow" id="MathJax-Span-483"><span class="mtext" id="MathJax-Span-484" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>j</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>j</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>Supply</mtext><mi>s</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>s</mi><mo>∈</mo><mtext>Sources</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-46" type="math/tex; mode=display">\begin{equation}
\sum_{j \in N(s)^+}{\text{flow}_{s,j}} \leq \text{Supply}_s \quad \forall s \in \text{Sources}
\tag{2}
\end{equation}</script><ul>
<li><strong>Pool capacity</strong>: Total outflow of pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-47-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-517" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-518"><span class="mi" id="MathJax-Span-519" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-47" type="math/tex">p</script> cannot exceed its capacity.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Cap&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-48-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-520" style="width: 100%; display: inline-block; min-width: 18.991em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.991em;"><span style="position: absolute; clip: rect(2.324em, 1014.82em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-521"><span class="mtable" id="MathJax-Span-522" style="min-width: 18.991em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.991em;"><span style="display: inline-block; position: absolute; width: 14.824em; height: 0px; clip: rect(-1.664em, 1014.82em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -7.438em;"><span style="position: absolute; clip: rect(2.324em, 1014.82em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 14.824em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1014.82em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -7.438em;"><span class="mtd" id="MathJax-Span-526"><span class="mrow" id="MathJax-Span-527"><span class="munderover" id="MathJax-Span-528"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-529" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-530"><span class="mrow" id="MathJax-Span-531"><span class="mi" id="MathJax-Span-532" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-533" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-534" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-535" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-536" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-537"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-538" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-539" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-540" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-541"><span class="msubsup" id="MathJax-Span-542"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-543" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-544"><span class="mrow" id="MathJax-Span-545"><span class="mi" id="MathJax-Span-546" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-547" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-548" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-549" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-550" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 2.205em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.73em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-551" style="font-family: MathJax_Main;">Cap</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 1.789em;"><span class="mi" id="MathJax-Span-552" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-553" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-554" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-555" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-556" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-557" style="font-family: MathJax_Main; padding-left: 0.301em;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-3"><span class="mrow" id="MathJax-Span-524"><span class="mtext" id="MathJax-Span-525" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3"><mtext>(3)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>Cap</mtext><mi>p</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Pools</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-48" type="math/tex; mode=display">\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{Cap}_p \quad \forall p \in \text{Pools}
\tag{3}
\end{equation}</script><ul>
<li><strong>Target demand</strong>: Total inflow of target <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-49-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-558" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-559"><span class="mi" id="MathJax-Span-560" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-49" type="math/tex">t</script> must at least meet its minimum demand.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-4"&gt;&lt;mtext&gt;(4)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Demand&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-50-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-561" style="width: 100%; display: inline-block; min-width: 21.372em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 21.372em;"><span style="position: absolute; clip: rect(2.324em, 1017.21em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-562"><span class="mtable" id="MathJax-Span-563" style="min-width: 21.372em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 21.372em;"><span style="display: inline-block; position: absolute; width: 17.265em; height: 0px; clip: rect(-1.664em, 1017.21em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -8.628em;"><span style="position: absolute; clip: rect(2.324em, 1017.21em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 17.265em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1017.21em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -8.628em;"><span class="mtd" id="MathJax-Span-567"><span class="mrow" id="MathJax-Span-568"><span class="munderover" id="MathJax-Span-569"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-570" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-571"><span class="mrow" id="MathJax-Span-572"><span class="mi" id="MathJax-Span-573" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-574" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-575" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-576" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-577" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-578"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-579" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-580" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-581" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-582"><span class="msubsup" id="MathJax-Span-583"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-584" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-585"><span class="mrow" id="MathJax-Span-586"><span class="mi" id="MathJax-Span-587" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-588" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-589" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-590" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-591" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.991em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.63em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-592" style="font-family: MathJax_Main;">Demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.634em;"><span class="mi" id="MathJax-Span-593" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-594" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-595" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-596" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-597" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-598" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-4"><span class="mrow" id="MathJax-Span-565"><span class="mtext" id="MathJax-Span-566" style="font-family: MathJax_Main;">(4)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-4"><mtext>(4)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≥</mo><msub><mtext>Demand</mtext><mi>t</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>t</mi><mo>∈</mo><mtext>Targets</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-50" type="math/tex; mode=display">\begin{equation}
\sum_{i \in N(t)^-}{\text{flow}_{i,t}} \geq \text{Demand}_t \quad \forall t \in \text{Targets}
\tag{4}
\end{equation}</script><ul>
<li><strong>Pool concentration</strong>: Concentration of attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-51-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-599" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-600"><span class="mi" id="MathJax-Span-601" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-51" type="math/tex">k</script> at pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-52-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-602" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-603"><span class="mi" id="MathJax-Span-604" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-52" type="math/tex">p</script> is expressed as the weighted average (linear blending) of the concentrations associated to the incoming flows (notice the bilinear terms on the right-hand side).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-5"&gt;&lt;mtext&gt;(5)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;quality&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Attrs&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-53-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-605" style="width: 100%; display: inline-block; min-width: 37.979em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 37.979em;"><span style="position: absolute; clip: rect(2.324em, 1033.75em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-606"><span class="mtable" id="MathJax-Span-607" style="min-width: 37.979em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 37.979em;"><span style="display: inline-block; position: absolute; width: 33.812em; height: 0px; clip: rect(-1.664em, 1033.75em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -16.902em;"><span style="position: absolute; clip: rect(2.324em, 1033.75em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 33.812em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1033.75em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -16.902em;"><span class="mtd" id="MathJax-Span-611"><span class="mrow" id="MathJax-Span-612"><span class="munderover" id="MathJax-Span-613"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-614" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.8em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-615"><span class="mrow" id="MathJax-Span-616"><span class="mi" id="MathJax-Span-617" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-618" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-619" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-620" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-621" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-622"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-623" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-624" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-625" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-626"><span class="msubsup" id="MathJax-Span-627"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.51em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-628" style="font-family: MathJax_Main;">Content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.574em;"><span class="texatom" id="MathJax-Span-629"><span class="mrow" id="MathJax-Span-630"><span class="mi" id="MathJax-Span-631" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-632" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-633" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-634" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-635" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-636" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-637"><span class="mrow" id="MathJax-Span-638"><span class="mi" id="MathJax-Span-639" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-640" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-641" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-642" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-643" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-644" style="font-family: MathJax_Main;">quality</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="texatom" id="MathJax-Span-645"><span class="mrow" id="MathJax-Span-646"><span class="mi" id="MathJax-Span-647" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-648" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-649" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-650" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-651" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-652" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-653"><span class="mrow" id="MathJax-Span-654"><span class="mi" id="MathJax-Span-655" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-656" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-657" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-658" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-659" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-660"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-661" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-662" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-663" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-664"><span class="msubsup" id="MathJax-Span-665"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-666" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-667"><span class="mrow" id="MathJax-Span-668"><span class="mi" id="MathJax-Span-669" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-670" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-671" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mspace" id="MathJax-Span-672" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-673" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-674" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-675" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-676" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-677" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-678" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-679" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-680" style="font-family: MathJax_Main; padding-left: 0.301em;">Pools</span><span class="mo" id="MathJax-Span-681" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-682" style="font-family: MathJax_Main; padding-left: 0.241em;">Attrs</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-5"><span class="mrow" id="MathJax-Span-609"><span class="mtext" id="MathJax-Span-610" style="font-family: MathJax_Main;">(5)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-5"><mtext>(5)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>=</mo><msub><mtext>quality</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>p</mi><mo>,</mo><mi>k</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Pools</mtext><mo>×</mo><mtext>Attrs</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-53" type="math/tex; mode=display">\begin{equation}
\sum_{s \in N(p)^-}{\text{Content}_{s,k} \cdot \text{flow}_{s,p}} = \text{quality}_{p,k} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}} \quad \forall (p,k) \in \text{Pools} \times \text{Attrs}
\tag{5}
\end{equation}</script><ul>
<li><strong>Target tolerances</strong>: Concentration of attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-54-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-683" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-684"><span class="mi" id="MathJax-Span-685" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-54" type="math/tex">k</script> at target <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-55-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-686" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-687"><span class="mi" id="MathJax-Span-688" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-55" type="math/tex">t</script> is also the result of linear blending, and must be within tolerances (notice the bilinear terms on the second expression of the left-hand side).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-6.1"&gt;&lt;mtext&gt;(6.1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;quality&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Min_tol&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Attrs&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-56-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-689" style="width: 100%; display: inline-block; min-width: 56.789em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 56.789em;"><span style="position: absolute; clip: rect(2.324em, 1051.02em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-690"><span class="mtable" id="MathJax-Span-691" style="min-width: 56.789em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 56.789em;"><span style="display: inline-block; position: absolute; width: 51.074em; height: 0px; clip: rect(-1.664em, 1051.02em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -25.533em;"><span style="position: absolute; clip: rect(2.324em, 1051.02em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 51.074em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1051.02em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -25.533em;"><span class="mtd" id="MathJax-Span-695"><span class="mrow" id="MathJax-Span-696"><span class="munderover" id="MathJax-Span-697"><span style="display: inline-block; position: relative; width: 5.479em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.027em;"><span class="mo" id="MathJax-Span-698" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1005.48em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-699"><span class="mrow" id="MathJax-Span-700"><span class="mi" id="MathJax-Span-701" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-702" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-703" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-704" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-705" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-706"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-707" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-708" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-709" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-710" style="font-size: 70.7%; font-family: MathJax_Main;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-711" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-712"><span class="msubsup" id="MathJax-Span-713"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.51em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-714" style="font-family: MathJax_Main;">Content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.574em;"><span class="texatom" id="MathJax-Span-715"><span class="mrow" id="MathJax-Span-716"><span class="mi" id="MathJax-Span-717" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-718" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-719" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-720" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-721" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-722" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-723"><span class="mrow" id="MathJax-Span-724"><span class="mi" id="MathJax-Span-725" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-726" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-727" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-728" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-729" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.729em;"><span class="mo" id="MathJax-Span-730" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1004.82em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-731"><span class="mrow" id="MathJax-Span-732"><span class="mi" id="MathJax-Span-733" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-734" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-735" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-736" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-737" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-738"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-739" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-740" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-741" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-742" style="font-size: 70.7%; font-family: MathJax_Main;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-743" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-744"><span class="msubsup" id="MathJax-Span-745"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-746" style="font-family: MathJax_Main;">quality</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="texatom" id="MathJax-Span-747"><span class="mrow" id="MathJax-Span-748"><span class="mi" id="MathJax-Span-749" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-750" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-751" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-752" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-753" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-754" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-755"><span class="mrow" id="MathJax-Span-756"><span class="mi" id="MathJax-Span-757" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-758" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-759" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-760" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-761" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.289em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.4em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-762" style="font-family: MathJax_Main;">Min_tol</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.396em;"><span class="texatom" id="MathJax-Span-763"><span class="mrow" id="MathJax-Span-764"><span class="mi" id="MathJax-Span-765" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-766" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-767" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-768" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-769" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-770" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-771"><span class="mrow" id="MathJax-Span-772"><span class="mi" id="MathJax-Span-773" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-774" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-775" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-776" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-777" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-778"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-779" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-780" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-781" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-782"><span class="msubsup" id="MathJax-Span-783"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-784" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-785"><span class="mrow" id="MathJax-Span-786"><span class="mi" id="MathJax-Span-787" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-788" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-789" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mspace" id="MathJax-Span-790" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-791" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-792" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-793" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-794" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-795" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-796" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-797" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-798" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span><span class="mo" id="MathJax-Span-799" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-800" style="font-family: MathJax_Main; padding-left: 0.241em;">Attrs</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 2.086em; height: 0px; clip: rect(-1.485em, 1001.97em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.97em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-6.1"><span class="mrow" id="MathJax-Span-693"><span class="mtext" id="MathJax-Span-694" style="font-family: MathJax_Main;">(6.1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-6.1"><mtext>(6.1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Sources</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Pools</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>quality</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≥</mo><msub><mtext>Min_tol</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>k</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Targets</mtext><mo>×</mo><mtext>Attrs</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-56" type="math/tex; mode=display">\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}}+ \sum_{p \in N(t)^- \cap \text{Pools}}{\text{quality}_{p,k} \cdot \text{flow}_{p,t}} \geq \text{Min_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \quad \forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{6.1}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-6.2"&gt;&lt;mtext&gt;(6.2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;quality&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Max_tol&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Attrs&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-57-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-801" style="width: 100%; display: inline-block; min-width: 56.967em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 56.967em;"><span style="position: absolute; clip: rect(2.324em, 1051.19em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-802"><span class="mtable" id="MathJax-Span-803" style="min-width: 56.967em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 56.967em;"><span style="display: inline-block; position: absolute; width: 51.253em; height: 0px; clip: rect(-1.664em, 1051.19em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -25.592em;"><span style="position: absolute; clip: rect(2.324em, 1051.19em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 51.253em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1051.19em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -25.592em;"><span class="mtd" id="MathJax-Span-807"><span class="mrow" id="MathJax-Span-808"><span class="munderover" id="MathJax-Span-809"><span style="display: inline-block; position: relative; width: 5.479em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.027em;"><span class="mo" id="MathJax-Span-810" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1005.48em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-811"><span class="mrow" id="MathJax-Span-812"><span class="mi" id="MathJax-Span-813" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-814" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-815" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-816" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-817" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-818"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-819" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-820" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-821" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-822" style="font-size: 70.7%; font-family: MathJax_Main;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-823" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-824"><span class="msubsup" id="MathJax-Span-825"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.51em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-826" style="font-family: MathJax_Main;">Content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.574em;"><span class="texatom" id="MathJax-Span-827"><span class="mrow" id="MathJax-Span-828"><span class="mi" id="MathJax-Span-829" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-830" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-831" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-832" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-833" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-834" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-835"><span class="mrow" id="MathJax-Span-836"><span class="mi" id="MathJax-Span-837" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-838" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-839" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-840" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-841" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.729em;"><span class="mo" id="MathJax-Span-842" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1004.82em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-843"><span class="mrow" id="MathJax-Span-844"><span class="mi" id="MathJax-Span-845" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-846" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-847" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-848" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-849" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-850"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-851" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-852" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-853" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-854" style="font-size: 70.7%; font-family: MathJax_Main;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-855" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-856"><span class="msubsup" id="MathJax-Span-857"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-858" style="font-family: MathJax_Main;">quality</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="texatom" id="MathJax-Span-859"><span class="mrow" id="MathJax-Span-860"><span class="mi" id="MathJax-Span-861" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-862" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-863" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-864" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-865" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-866" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-867"><span class="mrow" id="MathJax-Span-868"><span class="mi" id="MathJax-Span-869" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-870" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-871" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-872" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-873" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.57em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-874" style="font-family: MathJax_Main;">Max_tol</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.634em;"><span class="texatom" id="MathJax-Span-875"><span class="mrow" id="MathJax-Span-876"><span class="mi" id="MathJax-Span-877" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-878" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-879" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-880" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-881" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-882" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-883"><span class="mrow" id="MathJax-Span-884"><span class="mi" id="MathJax-Span-885" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-886" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-887" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-888" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-889" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-890"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-891" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-892" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-893" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-894"><span class="msubsup" id="MathJax-Span-895"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-896" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-897"><span class="mrow" id="MathJax-Span-898"><span class="mi" id="MathJax-Span-899" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-900" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-901" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mspace" id="MathJax-Span-902" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-903" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-904" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-905" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-906" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-907" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-908" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-909" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-910" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span><span class="mo" id="MathJax-Span-911" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-912" style="font-family: MathJax_Main; padding-left: 0.241em;">Attrs</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 2.086em; height: 0px; clip: rect(-1.485em, 1001.97em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.97em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-6.2"><span class="mrow" id="MathJax-Span-805"><span class="mtext" id="MathJax-Span-806" style="font-family: MathJax_Main;">(6.2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-6.2"><mtext>(6.2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Sources</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Pools</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>quality</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>Max_tol</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>k</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Targets</mtext><mo>×</mo><mtext>Attrs</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-57" type="math/tex; mode=display">\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}}+ \sum_{p \in N(t)^- \cap \text{Pools}}{\text{quality}_{p,k} \cdot \text{flow}_{p,t}} \leq \text{Max_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \quad \forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{6.2}
\end{equation}</script><p>The number of bilinear terms in this formulation is proportional to the number of attributes. An alternative formulation relies on decision variables that represent fractions of flow instead of concentrations, so that the bilinear terms are no longer associated to the number of attributes. Two types of decision variables may be used:</p>
<ul>
<li>fraction of total inflow at pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-58-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-913" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-914"><span class="mi" id="MathJax-Span-915" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-58" type="math/tex">p</script>, coming from source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-59-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-916" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-917"><span class="mi" id="MathJax-Span-918" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-59" type="math/tex">s</script>.</li>
<li>fraction of total outflow at pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-60-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-919" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-920"><span class="mi" id="MathJax-Span-921" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-60" type="math/tex">p</script>, going to terminal <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-61-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-922" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-923"><span class="mi" id="MathJax-Span-924" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-61" type="math/tex">t</script>.</li>
</ul>
<p>A model based on the first option is now presented:</p>
<h3 id="Q-formulation-(Proportion)">Q-formulation (Proportion)<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Q-formulation-(Proportion)">¶</a></h3><h4 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Decision-Variables">¶</a></h4><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;UB&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-62-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-925" style="width: 9.229em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1007.56em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-926"><span class="msubsup" id="MathJax-Span-927"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-928" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-929"><span class="mrow" id="MathJax-Span-930"><span class="mi" id="MathJax-Span-931" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-932" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-933" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-934" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-935" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-936" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-937" style="font-family: MathJax_Main;">,</span><span class="msubsup" id="MathJax-Span-938" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 2.265em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-939" style="font-family: MathJax_Main;">UB</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.432em;"><span class="texatom" id="MathJax-Span-940"><span class="mrow" id="MathJax-Span-941"><span class="mi" id="MathJax-Span-942" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-943" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-944" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-945" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><msub><mtext>UB</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-62" type="math/tex">\text{flow}_{i,j} \in [0, \text{UB}_{i,j}]</script>: Flow from node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-63-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-946" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-947"><span class="mi" id="MathJax-Span-948" style="font-family: MathJax_Math-italic;">i</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>i</mi></math></span></span><script id="MathJax-Element-63" type="math/tex">i</script> to node <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-64-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-949" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.42em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-950"><span class="mi" id="MathJax-Span-951" style="font-family: MathJax_Math-italic;">j</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>j</mi></math></span></span><script id="MathJax-Element-64" type="math/tex">j</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-65-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-952" style="width: 6.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.6em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-953"><span class="msubsup" id="MathJax-Span-954"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-955" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-956"><span class="mrow" id="MathJax-Span-957"><span class="mi" id="MathJax-Span-958" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-959" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-960" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-961" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-962" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-963"><span class="mrow" id="MathJax-Span-964"><span class="mi" id="MathJax-Span-965" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-966" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-65" type="math/tex">\text{prop}_{s,p} \in \mathbb{R}^+</script>: fraction of total inflow at pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-66-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-967" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-968"><span class="mi" id="MathJax-Span-969" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-66" type="math/tex">p</script>, coming from source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-67-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-970" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-971"><span class="mi" id="MathJax-Span-972" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-67" type="math/tex">s</script>.</p>
<p><strong>Note:</strong> The <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-68-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-973" style="width: 2.146em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.789em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1001.79em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-974"><span class="mtext" id="MathJax-Span-975" style="font-family: MathJax_Main;">flow</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>flow</mtext></math></span></span><script id="MathJax-Element-68" type="math/tex">\text{flow}</script> variables from sources to pools are replaced by the <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-69-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-976" style="width: 2.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.027em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.551em, 1001.97em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-977"><span class="mtext" id="MathJax-Span-978" style="font-family: MathJax_Main;">prop</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>prop</mtext></math></span></span><script id="MathJax-Element-69" type="math/tex">\text{prop}</script> variables.</p>
<h4 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Objective-Function">¶</a></h4><ul>
<li><strong>Profit</strong>: Maximize total profits (notice the bilinear terms on the second expression).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-0_1"&gt;&lt;mtext&gt;(0)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;mtext&gt;Max&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Price&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;cost&lt;/mtext&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;mrow&gt;&lt;mo&gt;(&lt;/mo&gt;&lt;mrow&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;mo&gt;)&lt;/mo&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-70-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-979" style="width: 100%; display: inline-block; min-width: 54.408em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 54.408em;"><span style="position: absolute; clip: rect(2.324em, 1049.94em, 6.253em, -999.997em); top: -4.521em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-980"><span class="mtable" id="MathJax-Span-981" style="min-width: 54.408em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 54.408em;"><span style="display: inline-block; position: absolute; width: 50.241em; height: 0px; clip: rect(-2.199em, 1049.94em, 1.729em, -999.997em); top: 0em; left: 50%; margin-left: -25.116em;"><span style="position: absolute; clip: rect(2.324em, 1049.94em, 6.253em, -999.997em); top: -4.521em; left: 0em;"><span style="display: inline-block; position: relative; width: 50.241em; height: 0px;"><span style="position: absolute; clip: rect(2.324em, 1049.94em, 6.253em, -999.997em); top: -4.521em; left: 50%; margin-left: -25.116em;"><span class="mtd" id="MathJax-Span-985"><span class="mrow" id="MathJax-Span-986"><span class="mtext" id="MathJax-Span-987" style="font-family: MathJax_Main;">Max</span><span class="mspace" id="MathJax-Span-988" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-989" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-990" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-991" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.098em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.836em;"><span class="mo" id="MathJax-Span-992" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.04em, 4.408em, -999.997em); top: -2.914em; left: 0em;"><span class="texatom" id="MathJax-Span-993"><span class="mrow" id="MathJax-Span-994"><span class="mi" id="MathJax-Span-995" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-996" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-997" style="font-size: 70.7%; font-family: MathJax_Main;">Targets</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-998" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-999"><span class="munderover" id="MathJax-Span-1000"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-1001" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1002"><span class="mrow" id="MathJax-Span-1003"><span class="mi" id="MathJax-Span-1004" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1005" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1006" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1007" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1008" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1009"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1010" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1011" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1012" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1013"><span class="msubsup" id="MathJax-Span-1014"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1002.21em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1015" style="font-family: MathJax_Main;">Price</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.265em;"><span class="mi" id="MathJax-Span-1016" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1017" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1018" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1019" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1020"><span class="mrow" id="MathJax-Span-1021"><span class="mi" id="MathJax-Span-1022" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1023" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1024" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1025" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-1026" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.158em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.836em;"><span class="mo" id="MathJax-Span-1027" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1003.1em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1028"><span class="mrow" id="MathJax-Span-1029"><span class="mi" id="MathJax-Span-1030" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1031" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-1032" style="font-size: 70.7%; font-family: MathJax_Main;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1033" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1034"><span class="msubsup" id="MathJax-Span-1035"><span style="display: inline-block; position: relative; width: 2.146em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1001.67em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1036" style="font-family: MathJax_Main;">cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.729em;"><span class="mi" id="MathJax-Span-1037" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1038" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="mrow" id="MathJax-Span-1039" style="padding-left: 0.241em;"><span class="mo" id="MathJax-Span-1040" style="vertical-align: 2.027em;"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; font-family: MathJax_Size4; top: -2.854em; left: 0em;">⎛<span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; font-family: MathJax_Size4; top: -1.068em; left: 0em;">⎝<span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mrow" id="MathJax-Span-1041"><span class="munderover" id="MathJax-Span-1042"><span style="display: inline-block; position: relative; width: 5.539em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.027em;"><span class="mo" id="MathJax-Span-1043" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1005.48em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1044"><span class="mrow" id="MathJax-Span-1045"><span class="mi" id="MathJax-Span-1046" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1047" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1048" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1049" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1050" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-1051"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1052" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1053" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1054" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1055" style="font-size: 70.7%; font-family: MathJax_Main;">Targets</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1056" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1057"><span class="msubsup" id="MathJax-Span-1058"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1059" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1060"><span class="mrow" id="MathJax-Span-1061"><span class="mi" id="MathJax-Span-1062" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1063" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1064" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1065" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-1066" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.943em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.729em;"><span class="mo" id="MathJax-Span-1067" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1004.88em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1068"><span class="mrow" id="MathJax-Span-1069"><span class="mi" id="MathJax-Span-1070" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1071" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1072" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1073" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1074" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-1075"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1076" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1077" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1078" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1079" style="font-size: 70.7%; font-family: MathJax_Main;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1080" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1081"><span class="msubsup" id="MathJax-Span-1082"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1083" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-1084"><span class="mrow" id="MathJax-Span-1085"><span class="mi" id="MathJax-Span-1086" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1087" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1088" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1089" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1090" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1091" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1092"><span class="mrow" id="MathJax-Span-1093"><span class="mi" id="MathJax-Span-1094" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1095" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1096" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1097" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1098" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1099"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1100" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1101" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1102" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1103"><span class="msubsup" id="MathJax-Span-1104"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1105" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1106"><span class="mrow" id="MathJax-Span-1107"><span class="mi" id="MathJax-Span-1108" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1109" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1110" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1111" style="vertical-align: 2.027em;"><span style="display: inline-block; position: relative; width: 0.896em; height: 0px;"><span style="position: absolute; font-family: MathJax_Size4; top: -2.854em; left: 0em;">⎞<span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; font-family: MathJax_Size4; top: -1.068em; left: 0em;">⎠<span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.527em;"></span></span></span><span style="display: inline-block; width: 0px; height: 4.527em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-0.949em, 1001.19em, 0.42em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -3.985em; right: 0em;"><span class="mtd" id="mjx-eqn-0_1"><span class="mrow" id="MathJax-Span-983"><span class="mtext" id="MathJax-Span-984" style="font-family: MathJax_Main;">(0)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.527em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.925em; border-left: 0px solid; width: 0px; height: 4.432em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-0_1"><mtext>(0)</mtext></mtd><mtd><mtext>Max</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Targets</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Price</mtext><mi>t</mi></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow></mrow><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mtext>Sources</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>cost</mtext><mi>s</mi></msub><mo>⋅</mo><mrow><mo>(</mo><mrow><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup><mo>∩</mo><mtext>Targets</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup><mo>∩</mo><mtext>Pools</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow></mrow></mrow><mo>)</mo></mrow></mrow></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-70" type="math/tex; mode=display">\begin{equation}
\text{Max} \quad Z = \sum_{t \in \text{Targets}}{\sum_{i \in N(t)^-}{\text{Price}_t \cdot \text{flow}_{i,t}}} - \sum_{s \in \text{Sources}}{\text{cost}_s \cdot \left( \sum_{t \in N(s)^+ \cap \text{Targets}}{\text{flow}_{s,t}} + \sum_{p \in N(s)^+ \cap \text{Pools}}{\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}}} \right) }
\tag{0}
\end{equation}</script><h4 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Constraints">¶</a></h4><ul>
<li><strong>Source capacity</strong>: Total outflow of source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-71-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1112" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1113"><span class="mi" id="MathJax-Span-1114" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-71" type="math/tex">s</script> cannot exceed its capacity (notice the bilinear terms on the first expression of the left-hand side).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-1_1"&gt;&lt;mtext&gt;(1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Supply&lt;/mtext&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-72-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1115" style="width: 100%; display: inline-block; min-width: 39.586em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 39.586em;"><span style="position: absolute; clip: rect(2.324em, 1035.36em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1116"><span class="mtable" id="MathJax-Span-1117" style="min-width: 39.586em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 39.586em;"><span style="display: inline-block; position: absolute; width: 35.42em; height: 0px; clip: rect(-1.664em, 1035.36em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -17.676em;"><span style="position: absolute; clip: rect(2.324em, 1035.36em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 35.42em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1035.36em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -17.676em;"><span class="mtd" id="MathJax-Span-1121"><span class="mrow" id="MathJax-Span-1122"><span class="munderover" id="MathJax-Span-1123"><span style="display: inline-block; position: relative; width: 4.943em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.729em;"><span class="mo" id="MathJax-Span-1124" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1004.88em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1125"><span class="mrow" id="MathJax-Span-1126"><span class="mi" id="MathJax-Span-1127" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1128" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1129" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1130" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1131" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-1132"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1133" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1134" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1135" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1136" style="font-size: 70.7%; font-family: MathJax_Main;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1137" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1138"><span class="msubsup" id="MathJax-Span-1139"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1140" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-1141"><span class="mrow" id="MathJax-Span-1142"><span class="mi" id="MathJax-Span-1143" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1144" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1145" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1146" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1147" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1148" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1149"><span class="mrow" id="MathJax-Span-1150"><span class="mi" id="MathJax-Span-1151" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1152" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1153" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1154" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1155" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1156"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1157" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1158" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1159" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1160"><span class="msubsup" id="MathJax-Span-1161"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1162" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1163"><span class="mrow" id="MathJax-Span-1164"><span class="mi" id="MathJax-Span-1165" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1166" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1167" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1168" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-1169" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 5.539em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.027em;"><span class="mo" id="MathJax-Span-1170" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1005.48em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1171"><span class="mrow" id="MathJax-Span-1172"><span class="mi" id="MathJax-Span-1173" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1174" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1175" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1176" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1177" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="msubsup" id="MathJax-Span-1178"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1179" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1180" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1181" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1182" style="font-size: 70.7%; font-family: MathJax_Main;">Targets</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1183" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1184"><span class="msubsup" id="MathJax-Span-1185"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1186" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1187"><span class="mrow" id="MathJax-Span-1188"><span class="mi" id="MathJax-Span-1189" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1190" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1191" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1192" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-1193" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.455em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.04em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1194" style="font-family: MathJax_Main;">Supply</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.039em;"><span class="mi" id="MathJax-Span-1195" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1196" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1197" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1198" style="font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1199" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1200" style="font-family: MathJax_Main; padding-left: 0.301em;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-1_1"><span class="mrow" id="MathJax-Span-1119"><span class="mtext" id="MathJax-Span-1120" style="font-family: MathJax_Main;">(1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-1_1"><mtext>(1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup><mo>∩</mo><mtext>Pools</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>s</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup><mo>∩</mo><mtext>Targets</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>Supply</mtext><mi>s</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>s</mi><mo>∈</mo><mtext>Sources</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-72" type="math/tex; mode=display">\begin{equation}
\sum_{p \in N(s)^+ \cap \text{Pools}}{\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}}} + \sum_{t \in N(s)^+ \cap \text{Targets}}{\text{flow}_{s,t}} \leq \text{Supply}_s \quad \forall s \in \text{Sources}
\tag{1}
\end{equation}</script><ul>
<li><strong>Pool capacity</strong>: Total outflow of pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-73-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1201" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1202"><span class="mi" id="MathJax-Span-1203" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-73" type="math/tex">p</script> cannot exceed its capacity.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-2_1"&gt;&lt;mtext&gt;(2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Cap&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-74-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1204" style="width: 100%; display: inline-block; min-width: 18.991em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 18.991em;"><span style="position: absolute; clip: rect(2.324em, 1014.82em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1205"><span class="mtable" id="MathJax-Span-1206" style="min-width: 18.991em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 18.991em;"><span style="display: inline-block; position: absolute; width: 14.824em; height: 0px; clip: rect(-1.664em, 1014.82em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -7.438em;"><span style="position: absolute; clip: rect(2.324em, 1014.82em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 14.824em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1014.82em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -7.438em;"><span class="mtd" id="MathJax-Span-1210"><span class="mrow" id="MathJax-Span-1211"><span class="munderover" id="MathJax-Span-1212"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1213" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1214"><span class="mrow" id="MathJax-Span-1215"><span class="mi" id="MathJax-Span-1216" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1217" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1218" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1219" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1220" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1221"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1222" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1223" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1224" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1225"><span class="msubsup" id="MathJax-Span-1226"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1227" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1228"><span class="mrow" id="MathJax-Span-1229"><span class="mi" id="MathJax-Span-1230" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1231" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1232" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1233" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-1234" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 2.205em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.73em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1235" style="font-family: MathJax_Main;">Cap</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 1.789em;"><span class="mi" id="MathJax-Span-1236" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1237" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1238" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1239" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1240" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1241" style="font-family: MathJax_Main; padding-left: 0.301em;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-2_1"><span class="mrow" id="MathJax-Span-1208"><span class="mtext" id="MathJax-Span-1209" style="font-family: MathJax_Main;">(2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-2_1"><mtext>(2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>Cap</mtext><mi>p</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Pools</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-74" type="math/tex; mode=display">\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{Cap}_p \quad \forall p \in \text{Pools}
\tag{2}
\end{equation}</script><ul>
<li><strong>Target demand</strong>: Total inflow of target <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-75-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1242" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1243"><span class="mi" id="MathJax-Span-1244" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-75" type="math/tex">t</script> must at least meet its minimum demand.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-3_1"&gt;&lt;mtext&gt;(3)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Demand&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-76-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1245" style="width: 100%; display: inline-block; min-width: 21.372em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 21.372em;"><span style="position: absolute; clip: rect(2.324em, 1017.21em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1246"><span class="mtable" id="MathJax-Span-1247" style="min-width: 21.372em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 21.372em;"><span style="display: inline-block; position: absolute; width: 17.265em; height: 0px; clip: rect(-1.664em, 1017.21em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -8.628em;"><span style="position: absolute; clip: rect(2.324em, 1017.21em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 17.265em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1017.21em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -8.628em;"><span class="mtd" id="MathJax-Span-1251"><span class="mrow" id="MathJax-Span-1252"><span class="munderover" id="MathJax-Span-1253"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-1254" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1255"><span class="mrow" id="MathJax-Span-1256"><span class="mi" id="MathJax-Span-1257" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1258" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1259" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1260" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1261" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1262"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1263" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1264" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1265" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1266"><span class="msubsup" id="MathJax-Span-1267"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1268" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1269"><span class="mrow" id="MathJax-Span-1270"><span class="mi" id="MathJax-Span-1271" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1272" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1273" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1274" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1275" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.991em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.63em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1276" style="font-family: MathJax_Main;">Demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.634em;"><span class="mi" id="MathJax-Span-1277" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1278" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1279" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1280" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1281" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1282" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-3_1"><span class="mrow" id="MathJax-Span-1249"><span class="mtext" id="MathJax-Span-1250" style="font-family: MathJax_Main;">(3)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-3_1"><mtext>(3)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≥</mo><msub><mtext>Demand</mtext><mi>t</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>t</mi><mo>∈</mo><mtext>Targets</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-76" type="math/tex; mode=display">\begin{equation}
\sum_{i \in N(t)^-}{\text{flow}_{i,t}} \geq \text{Demand}_t \quad \forall t \in \text{Targets}
\tag{3}
\end{equation}</script><ul>
<li><strong>Pool inflow</strong>: The sum of the contributions of all incoming sources to pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-77-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1283" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1284"><span class="mi" id="MathJax-Span-1285" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-77" type="math/tex">p</script> must equal one.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-4_1"&gt;&lt;mtext&gt;(4)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-78-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1286" style="width: 100%; display: inline-block; min-width: 17.622em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 17.622em;"><span style="position: absolute; clip: rect(2.324em, 1013.46em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1287"><span class="mtable" id="MathJax-Span-1288" style="min-width: 17.622em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 17.622em;"><span style="display: inline-block; position: absolute; width: 13.515em; height: 0px; clip: rect(-1.664em, 1013.46em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -6.723em;"><span style="position: absolute; clip: rect(2.324em, 1013.46em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 13.515em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1013.46em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -6.723em;"><span class="mtd" id="MathJax-Span-1292"><span class="mrow" id="MathJax-Span-1293"><span class="munderover" id="MathJax-Span-1294"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1295" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.8em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1296"><span class="mrow" id="MathJax-Span-1297"><span class="mi" id="MathJax-Span-1298" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1299" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1300" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1301" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1302" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1303"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1304" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1305" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1306" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1307"><span class="msubsup" id="MathJax-Span-1308"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1309" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-1310"><span class="mrow" id="MathJax-Span-1311"><span class="mi" id="MathJax-Span-1312" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1313" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1314" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1315" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-1316" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span><span class="mspace" id="MathJax-Span-1317" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1318" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1319" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1320" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1321" style="font-family: MathJax_Main; padding-left: 0.301em;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-4_1"><span class="mrow" id="MathJax-Span-1290"><span class="mtext" id="MathJax-Span-1291" style="font-family: MathJax_Main;">(4)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-4_1"><mtext>(4)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>=</mo><mn>1</mn><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Pools</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-78" type="math/tex; mode=display">\begin{equation}
\sum_{s \in N(p)^-}{\text{prop}_{s,p}} = 1 \quad \forall p \in \text{Pools}
\tag{4}
\end{equation}</script><ul>
<li><strong>Target tolerances</strong>: Concentration of attribute <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-79-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1322" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1323"><span class="mi" id="MathJax-Span-1324" style="font-family: MathJax_Math-italic;">k</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>k</mi></math></span></span><script id="MathJax-Element-79" type="math/tex">k</script> at target <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-80-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1325" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1326"><span class="mi" id="MathJax-Span-1327" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-80" type="math/tex">t</script> is also the result of linear blending, and must be within tolerances (notice the bilinear terms on the second expression of the left-hand side).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-5.1"&gt;&lt;mtext&gt;(5.1)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Min_tol&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace linebreak="newline" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Attrs&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-81-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1328" style="width: 100%; display: inline-block; min-width: 51.789em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 51.789em;"><span style="position: absolute; clip: rect(2.503em, 1046.07em, 6.789em, -999.997em); top: -4.878em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1329"><span class="mtable" id="MathJax-Span-1330" style="min-width: 51.789em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 51.789em;"><span style="display: inline-block; position: absolute; width: 46.074em; height: 0px; clip: rect(-2.378em, 1046.07em, 1.908em, -999.997em); top: 0em; left: 50%; margin-left: -23.033em;"><span style="position: absolute; clip: rect(2.503em, 1046.07em, 6.789em, -999.997em); top: -4.878em; left: 0em;"><span style="display: inline-block; position: relative; width: 46.074em; height: 0px;"><span style="position: absolute; width: 100%; clip: rect(3.753em, 1046.07em, 8.039em, -999.997em); top: -6.128em; left: 0em;"><span class="mtd" id="MathJax-Span-1334"><span class="mrow" id="MathJax-Span-1335"><span style="display: inline-block; position: relative; width: 100%; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1046.07em, 5.717em, -999.997em); top: -3.985em; left: 50%; margin-left: -23.033em;"><span class="munderover" id="MathJax-Span-1336"><span style="display: inline-block; position: relative; width: 5.479em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.027em;"><span class="mo" id="MathJax-Span-1337" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1005.48em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1338"><span class="mrow" id="MathJax-Span-1339"><span class="mi" id="MathJax-Span-1340" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1341" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1342" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1343" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1344" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1345"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1346" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1347" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1348" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1349" style="font-size: 70.7%; font-family: MathJax_Main;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1350" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1351"><span class="msubsup" id="MathJax-Span-1352"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.51em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1353" style="font-family: MathJax_Main;">Content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.574em;"><span class="texatom" id="MathJax-Span-1354"><span class="mrow" id="MathJax-Span-1355"><span class="mi" id="MathJax-Span-1356" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1357" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1358" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1359" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1360" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1361" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1362"><span class="mrow" id="MathJax-Span-1363"><span class="mi" id="MathJax-Span-1364" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1365" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1366" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1367" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-1368" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.729em;"><span class="mo" id="MathJax-Span-1369" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1004.82em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1370"><span class="mrow" id="MathJax-Span-1371"><span class="mi" id="MathJax-Span-1372" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1373" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1374" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1375" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1376" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1377"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1378" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1379" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1380" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1381" style="font-size: 70.7%; font-family: MathJax_Main;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1382" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1383"><span class="msubsup" id="MathJax-Span-1384"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1385" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1386"><span class="mrow" id="MathJax-Span-1387"><span class="mi" id="MathJax-Span-1388" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1389" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1390" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1391" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1392" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1393" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.8em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1394"><span class="mrow" id="MathJax-Span-1395"><span class="mi" id="MathJax-Span-1396" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1397" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1398" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1399" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1400" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1401"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1402" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1403" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1404" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1405"><span class="msubsup" id="MathJax-Span-1406"><span style="display: inline-block; position: relative; width: 4.229em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1003.22em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1407" style="font-family: MathJax_Main;">content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.277em;"><span class="texatom" id="MathJax-Span-1408"><span class="mrow" id="MathJax-Span-1409"><span class="mi" id="MathJax-Span-1410" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1411" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1412" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1413" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1414" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1415" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-1416"><span class="mrow" id="MathJax-Span-1417"><span class="mi" id="MathJax-Span-1418" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1419" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1420" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1421" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-1422" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.289em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.4em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1423" style="font-family: MathJax_Main;">Min_tol</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.396em;"><span class="texatom" id="MathJax-Span-1424"><span class="mrow" id="MathJax-Span-1425"><span class="mi" id="MathJax-Span-1426" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1427" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1428" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1429" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1430" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-1431" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1432"><span class="mrow" id="MathJax-Span-1433"><span class="mi" id="MathJax-Span-1434" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1435" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1436" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1437" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1438" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1439"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1440" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1441" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1442" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1443"><span class="msubsup" id="MathJax-Span-1444"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1445" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1446"><span class="mrow" id="MathJax-Span-1447"><span class="mi" id="MathJax-Span-1448" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1449" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1450" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.039em, 1010.84em, 4.408em, -999.997em); top: -1.307em; left: 50%; margin-left: -5.414em;"><span class="mspace" id="MathJax-Span-1451" style="height: 0em; vertical-align: 0em; width: 0em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1452" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-1453" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1454" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1455" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1456" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-1457" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-1458" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1459" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span><span class="mo" id="MathJax-Span-1460" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-1461" style="font-family: MathJax_Main; padding-left: 0.241em;">Attrs</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; position: absolute; width: 2.086em; height: 0px; clip: rect(-2.199em, 1001.97em, -0.83em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.97em, 4.408em, -999.997em); top: -5.235em; right: 0em;"><span class="mtd" id="mjx-eqn-5.1"><span class="mrow" id="MathJax-Span-1332"><span class="mtext" id="MathJax-Span-1333" style="font-family: MathJax_Main;">(5.1)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -2.139em; border-left: 0px solid; width: 0px; height: 4.861em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-5.1"><mtext>(5.1)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Sources</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Pools</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></mrow><mo>≥</mo><msub><mtext>Min_tol</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mspace linebreak="newline"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>k</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Targets</mtext><mo>×</mo><mtext>Attrs</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-81" type="math/tex; mode=display">\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}} + \sum_{p \in N(t)^- \cap \text{Pools}}{\text{flow}_{p,t} \cdot \sum_{s \in N(p)^-}{\text{content}_{s,k} \cdot \text{prop}_{s,p}}}
\geq \text{Min_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \\
\forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{5.1}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-5.2"&gt;&lt;mtext&gt;(5.2)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;Content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;content&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;Max_tol&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace linebreak="newline" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;k&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Targets&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Attrs&lt;/mtext&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-82-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1462" style="width: 100%; display: inline-block; min-width: 51.967em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 51.967em;"><span style="position: absolute; clip: rect(2.503em, 1046.25em, 6.789em, -999.997em); top: -4.878em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1463"><span class="mtable" id="MathJax-Span-1464" style="min-width: 51.967em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 51.967em;"><span style="display: inline-block; position: absolute; width: 46.253em; height: 0px; clip: rect(-2.378em, 1046.25em, 1.908em, -999.997em); top: 0em; left: 50%; margin-left: -23.152em;"><span style="position: absolute; clip: rect(2.503em, 1046.25em, 6.789em, -999.997em); top: -4.878em; left: 0em;"><span style="display: inline-block; position: relative; width: 46.253em; height: 0px;"><span style="position: absolute; width: 100%; clip: rect(3.753em, 1046.25em, 8.039em, -999.997em); top: -6.128em; left: 0em;"><span class="mtd" id="MathJax-Span-1468"><span class="mrow" id="MathJax-Span-1469"><span style="display: inline-block; position: relative; width: 100%; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1046.25em, 5.717em, -999.997em); top: -3.985em; left: 50%; margin-left: -23.152em;"><span class="munderover" id="MathJax-Span-1470"><span style="display: inline-block; position: relative; width: 5.479em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.027em;"><span class="mo" id="MathJax-Span-1471" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1005.48em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1472"><span class="mrow" id="MathJax-Span-1473"><span class="mi" id="MathJax-Span-1474" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1475" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1476" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1477" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1478" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1479"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1480" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1481" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1482" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1483" style="font-size: 70.7%; font-family: MathJax_Main;">Sources</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1484" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1485"><span class="msubsup" id="MathJax-Span-1486"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.51em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1487" style="font-family: MathJax_Main;">Content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.574em;"><span class="texatom" id="MathJax-Span-1488"><span class="mrow" id="MathJax-Span-1489"><span class="mi" id="MathJax-Span-1490" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1491" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1492" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1493" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1494" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1495" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1496"><span class="mrow" id="MathJax-Span-1497"><span class="mi" id="MathJax-Span-1498" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1499" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1500" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1501" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-1502" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.729em;"><span class="mo" id="MathJax-Span-1503" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1004.82em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1504"><span class="mrow" id="MathJax-Span-1505"><span class="mi" id="MathJax-Span-1506" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1507" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1508" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1509" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1510" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1511"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1512" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1513" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1514" style="font-size: 70.7%; font-family: MathJax_Main;">∩</span><span class="mtext" id="MathJax-Span-1515" style="font-size: 70.7%; font-family: MathJax_Main;">Pools</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1516" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1517"><span class="msubsup" id="MathJax-Span-1518"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1519" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1520"><span class="mrow" id="MathJax-Span-1521"><span class="mi" id="MathJax-Span-1522" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1523" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1524" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1525" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1526" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1527" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.8em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1528"><span class="mrow" id="MathJax-Span-1529"><span class="mi" id="MathJax-Span-1530" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1531" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1532" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1533" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1534" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1535"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1536" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1537" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1538" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1539"><span class="msubsup" id="MathJax-Span-1540"><span style="display: inline-block; position: relative; width: 4.229em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1003.22em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1541" style="font-family: MathJax_Main;">content</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.277em;"><span class="texatom" id="MathJax-Span-1542"><span class="mrow" id="MathJax-Span-1543"><span class="mi" id="MathJax-Span-1544" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1545" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1546" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1547" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="msubsup" id="MathJax-Span-1548" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1549" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-1550"><span class="mrow" id="MathJax-Span-1551"><span class="mi" id="MathJax-Span-1552" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1553" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1554" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1555" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-1556" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.527em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.57em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1557" style="font-family: MathJax_Main;">Max_tol</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.634em;"><span class="texatom" id="MathJax-Span-1558"><span class="mrow" id="MathJax-Span-1559"><span class="mi" id="MathJax-Span-1560" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1561" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1562" style="font-size: 70.7%; font-family: MathJax_Math-italic;">k</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1563" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1564" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-1565" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.62em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1566"><span class="mrow" id="MathJax-Span-1567"><span class="mi" id="MathJax-Span-1568" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1569" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1570" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1571" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1572" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="msubsup" id="MathJax-Span-1573"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1574" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1575" style="font-size: 50%; font-family: MathJax_Main;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1576" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1577"><span class="msubsup" id="MathJax-Span-1578"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1579" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1580"><span class="mrow" id="MathJax-Span-1581"><span class="mi" id="MathJax-Span-1582" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1583" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1584" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.039em, 1010.84em, 4.408em, -999.997em); top: -1.307em; left: 50%; margin-left: -5.414em;"><span class="mspace" id="MathJax-Span-1585" style="height: 0em; vertical-align: 0em; width: 0em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1586" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-1587" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1588" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1589" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1590" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">k</span><span class="mo" id="MathJax-Span-1591" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-1592" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1593" style="font-family: MathJax_Main; padding-left: 0.301em;">Targets</span><span class="mo" id="MathJax-Span-1594" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-1595" style="font-family: MathJax_Main; padding-left: 0.241em;">Attrs</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; position: absolute; width: 2.086em; height: 0px; clip: rect(-2.199em, 1001.97em, -0.83em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.97em, 4.408em, -999.997em); top: -5.235em; right: 0em;"><span class="mtd" id="mjx-eqn-5.2"><span class="mrow" id="MathJax-Span-1466"><span class="mtext" id="MathJax-Span-1467" style="font-family: MathJax_Main;">(5.2)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -2.139em; border-left: 0px solid; width: 0px; height: 4.861em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-5.2"><mtext>(5.2)</mtext></mtd><mtd><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Sources</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>Content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup><mo>∩</mo><mtext>Pools</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>content</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></mrow><mo>≤</mo><msub><mtext>Max_tol</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>k</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>t</mi><msup><mo stretchy="false">)</mo><mo>−</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mspace linebreak="newline"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>k</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Targets</mtext><mo>×</mo><mtext>Attrs</mtext></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-82" type="math/tex; mode=display">\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}} + \sum_{p \in N(t)^- \cap \text{Pools}}{\text{flow}_{p,t} \cdot \sum_{s \in N(p)^-}{\text{content}_{s,k} \cdot \text{prop}_{s,p}}}
\leq \text{Max_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \\
\forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{5.2}
\end{equation}</script><p>One drawback is that if some of the source-to-pool edges have flow capacity, we need to define additional constraints instead of just specifying upper bounds on the associated decision variables. Such constraints can be defined with bilinear terms as follows:</p>
<ul>
<li><strong>Flow limit</strong>: Flow from source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-83-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1596" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1597"><span class="mi" id="MathJax-Span-1598" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-83" type="math/tex">s</script> to pool <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-84-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1599" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1600"><span class="mi" id="MathJax-Span-1601" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-84" type="math/tex">p</script> cannot exceed the installed capacity (notice the bilinear terms on the left-hand side). In the P-formulation, declaring this is as easy as setting the upper bound of the associated <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-85-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1602" style="width: 2.146em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.789em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1001.79em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1603"><span class="mtext" id="MathJax-Span-1604" style="font-family: MathJax_Main;">flow</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>flow</mtext></math></span></span><script id="MathJax-Element-85" type="math/tex">\text{flow}</script> variable. However, this variable no longer exists in the Q-formulation so we need to model the capacity as a constraint.  </li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtable displaystyle="true"&gt;&lt;mlabeledtr&gt;&lt;mtd id="mjx-eqn-6"&gt;&lt;mtext&gt;(6)&lt;/mtext&gt;&lt;/mtd&gt;&lt;mtd&gt;&lt;msub&gt;&lt;mtext&gt;prop&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x22C5;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;N&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;msup&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;UB&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mi&gt;E&lt;/mi&gt;&lt;mo&gt;&amp;#x2229;&lt;/mo&gt;&lt;mrow&gt;&lt;mo&gt;(&lt;/mo&gt;&lt;mrow&gt;&lt;mtext&gt;Sources&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Pools&lt;/mtext&gt;&lt;/mrow&gt;&lt;mo&gt;)&lt;/mo&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2223;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;UB&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;j&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;lt;&lt;/mo&gt;&lt;mi mathvariant="normal"&gt;&amp;#x221E;&lt;/mi&gt;&lt;/mtd&gt;&lt;/mlabeledtr&gt;&lt;/mtable&gt;&lt;/math&gt;' id="MathJax-Element-86-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1605" style="width: 100%; display: inline-block; min-width: 37.205em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%; min-width: 37.205em;"><span style="position: absolute; clip: rect(2.324em, 1032.98em, 5.182em, -999.997em); top: -3.985em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1606"><span class="mtable" id="MathJax-Span-1607" style="min-width: 37.205em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; min-width: 37.205em;"><span style="display: inline-block; position: absolute; width: 33.039em; height: 0px; clip: rect(-1.664em, 1032.98em, 1.193em, -999.997em); top: 0em; left: 50%; margin-left: -16.485em;"><span style="position: absolute; clip: rect(2.324em, 1032.98em, 5.182em, -999.997em); top: -3.985em; left: 0em;"><span style="display: inline-block; position: relative; width: 33.039em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1032.98em, 5.717em, -999.997em); top: -4.521em; left: 50%; margin-left: -16.485em;"><span class="mtd" id="MathJax-Span-1611"><span class="mrow" id="MathJax-Span-1612"><span class="msubsup" id="MathJax-Span-1613"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1001.97em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1614" style="font-family: MathJax_Main;">prop</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.027em;"><span class="texatom" id="MathJax-Span-1615"><span class="mrow" id="MathJax-Span-1616"><span class="mi" id="MathJax-Span-1617" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1618" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1619" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1620" style="font-family: MathJax_Main; padding-left: 0.241em;">⋅</span><span class="munderover" id="MathJax-Span-1621" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.658em;"><span class="mo" id="MathJax-Span-1622" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.158em, 1002.68em, 4.467em, -999.997em); top: -2.735em; left: 0em;"><span class="texatom" id="MathJax-Span-1623"><span class="mrow" id="MathJax-Span-1624"><span class="mi" id="MathJax-Span-1625" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-1626" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mi" id="MathJax-Span-1627" style="font-size: 70.7%; font-family: MathJax_Math-italic;">N<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-1628" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1629" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="msubsup" id="MathJax-Span-1630"><span style="display: inline-block; position: relative; width: 0.717em; height: 0px;"><span style="position: absolute; clip: rect(3.277em, 1000.18em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mo" id="MathJax-Span-1631" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.342em; left: 0.301em;"><span class="mo" id="MathJax-Span-1632" style="font-size: 50%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1633" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1634"><span class="msubsup" id="MathJax-Span-1635"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1636" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-1637"><span class="mrow" id="MathJax-Span-1638"><span class="mi" id="MathJax-Span-1639" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1640" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1641" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-1642" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-1643" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1644" style="font-family: MathJax_Main;">UB</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.432em;"><span class="texatom" id="MathJax-Span-1645"><span class="mrow" id="MathJax-Span-1646"><span class="mi" id="MathJax-Span-1647" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-1648" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1649" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-1650" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1651" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-1652" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-1653" style="font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1654" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1655" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">j</span><span class="mo" id="MathJax-Span-1656" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-1657" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mi" id="MathJax-Span-1658" style="font-family: MathJax_Math-italic; padding-left: 0.301em;">E<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-1659" style="font-family: MathJax_Main; padding-left: 0.241em;">∩</span><span class="mrow" id="MathJax-Span-1660" style="padding-left: 0.241em;"><span class="mo" id="MathJax-Span-1661" style="font-family: MathJax_Main;">(</span><span class="mrow" id="MathJax-Span-1662"><span class="mtext" id="MathJax-Span-1663" style="font-family: MathJax_Main;">Sources</span><span class="mo" id="MathJax-Span-1664" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-1665" style="font-family: MathJax_Main; padding-left: 0.241em;">Pools</span></span><span class="mo" id="MathJax-Span-1666" style="font-family: MathJax_Main;">)</span></span><span class="mo" id="MathJax-Span-1667" style="font-family: MathJax_Main; padding-left: 0.301em;">∣</span><span class="msubsup" id="MathJax-Span-1668" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 2.265em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1001.43em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1669" style="font-family: MathJax_Main;">UB</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.432em;"><span class="texatom" id="MathJax-Span-1670"><span class="mrow" id="MathJax-Span-1671"><span class="mi" id="MathJax-Span-1672" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mo" id="MathJax-Span-1673" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1674" style="font-size: 70.7%; font-family: MathJax_Math-italic;">j</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1675" style="font-family: MathJax_Main; padding-left: 0.301em;">&lt;</span><span class="mi" id="MathJax-Span-1676" style="font-family: MathJax_Main; padding-left: 0.301em;">∞</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; position: absolute; width: 1.253em; height: 0px; clip: rect(-1.485em, 1001.19em, -0.116em, -999.997em); top: 0em; right: 0em; margin-right: 0em;"><span style="position: absolute; clip: rect(3.039em, 1001.19em, 4.408em, -999.997em); top: -4.521em; right: 0em;"><span class="mtd" id="mjx-eqn-6"><span class="mrow" id="MathJax-Span-1609"><span class="mtext" id="MathJax-Span-1610" style="font-family: MathJax_Main;">(6)</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.282em; border-left: 0px solid; width: 0px; height: 3.146em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtable displaystyle="true"><mlabeledtr><mtd id="mjx-eqn-6"><mtext>(6)</mtext></mtd><mtd><msub><mtext>prop</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>⋅</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mi>N</mi><mo stretchy="false">(</mo><mi>p</mi><msup><mo stretchy="false">)</mo><mo>+</mo></msup></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>UB</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>i</mi><mo>,</mo><mi>j</mi><mo stretchy="false">)</mo><mo>∈</mo><mi>E</mi><mo>∩</mo><mrow><mo>(</mo><mrow><mtext>Sources</mtext><mo>×</mo><mtext>Pools</mtext></mrow><mo>)</mo></mrow><mo>∣</mo><msub><mtext>UB</mtext><mrow class="MJX-TeXAtom-ORD"><mi>i</mi><mo>,</mo><mi>j</mi></mrow></msub><mo>&lt;</mo><mi mathvariant="normal">∞</mi></mtd></mlabeledtr></mtable></math></span></span></div><script id="MathJax-Element-86" type="math/tex; mode=display">\begin{equation}
\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{UB}_{s,p} \quad \forall (i,j) \in E \cap \left( \text{Sources} \times \text{Pools} \right) \mid \text{UB}_{i,j} < \infty
\tag{6}
\end{equation}</script><hr/>
<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Python-Implementation">¶</a></h2><p>Solving Bilinear Programs with Gurobi is as easy as configuring the global parameter <code>nonConvex</code>. When setting this parameter to a value of 2, non-convex quadratic problems are solved by means of translating them into bilinear form and applying sB&amp;B. We will first deploy the P-formulation model, and then compare it with the Q-formulation model:</p>
<h3 id="P-formulation-(Concentration)">P-formulation (Concentration)<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#P-formulation-(Concentration)">¶</a></h3>


In [ ]:

p_pooling = gp.Model("Pooling")

# Set global parameters
p_pooling.params.nonConvex = 2
p_pooling.params.timelimit = 5*60 # time limit of 5 minutes

# Declare decision variables

# flow
ik = p_pooling.addVars(s2t, name="Source2Target")
ij = p_pooling.addVars(s2p, name="Source2Pool")
jk = p_pooling.addVars(p2t, name="Pool2Target")
ik["s1","t2"].ub = 750
ik["s3","t1"].ub = 750
# quality
prop = p_pooling.addVars(pools, attrs, name="Proportion")

# Deploy constraint sets

# 1. Flow conservation
p_pooling.addConstrs((ij.sum('*',j) == jk.sum(j,'*') for j in pools),
                     name="Flow_conservation")
# 2. Source capacity
p_pooling.addConstrs((ij.sum(i,'*') + ik.sum(i,'*') <= supply[i] for i in sources),
                     name="Source_capacity")
# 3. Pool capacity
p_pooling.addConstrs((jk.sum(j,'*') <= cap[j] for j in pools),
                     name="Pool_capacity")
# 4. Target demand
p_pooling.addConstrs((ik.sum('*',k) + jk.sum('*',k) >= demand[k] for k in targets),
                     name="Target_demand")
# 5. Pool concentration
p_pooling.addConstrs((gp.quicksum(content[i][attr]*ij[i,j]
                               for i in sources if (i,j) in s2p)
                      == prop[j,attr]*jk.sum(j,'*') for j in pools for attr in attrs),
                     name="Pool_concentration")
# 6.1 Target (min) tolerances
p_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(prop[j,attr]*jk[j,k]
                                 for j in pools if (j,k) in p2t)
                      >= min_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in min_tol[k].keys()),
                     name="Target_min_tolerances")
# 6.2 Target (max) tolerances
p_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(prop[j,attr]*jk[j,k]
                                 for j in pools if (j,k) in p2t)
                      <= max_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in max_tol[k].keys()),
                     name="Target_max_tolerances")

# Deploy Objective Function

# 0. Total profit
obj = gp.quicksum(price[k]*(ik.sum('*',k) + jk.sum('*',k))
               for k in targets) \
- gp.quicksum(cost[i]*(ij.sum(i,'*') + ik.sum(i,'*'))
           for i in sources)
p_pooling.setObjective(obj, GRB.MAXIMIZE)

# Find the optimal solution
p_pooling.optimize()




<p>The P-formulation for this instance has:</p>
<ul>
<li>24 decision variables.</li>
<li>10 linear constraints.</li>
<li>20 bilinear constraints.</li>
<li>a linear objective function.</li>
</ul>
<p>As can be seen, we observe a gap of 103.23% after reaching the time limit of five minutes (at this point, the incumbent solution induces a total profit of 411,530.70 USD). In fact, even after 20 minutes the solver does not make much progress in closing the gap.</p>
<h3 id="Q-Formulation-(proportion)">Q-Formulation (proportion)<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Q-Formulation-(proportion)">¶</a></h3><p>Let's now see how the q-formulation model performs:</p>


In [ ]:

q_pooling = gp.Model("Pooling")

# Set global parameters
q_pooling.params.nonConvex = 2
q_pooling.params.timelimit = 5*60

# Declare decision variables

# flow
ik = q_pooling.addVars(s2t, name="Source2Target")
jk = q_pooling.addVars(p2t, name="Pool2Target")
ik["s1","t2"].ub = 750
ik["s3","t1"].ub = 750
# proportion
p_ij = q_pooling.addVars(s2p, ub=1.0, name="Prop_Source2Pool")

# Deploy constraint sets

# 1. Source capacity
q_pooling.addConstrs((gp.quicksum(p_ij[i,j]*jk.sum(j,'*')
                               for j in pools if (i,j) in s2p)
                      + ik.sum(i,'*') <= supply[i] for i in sources),
                     name="Source_capacity")
# 2. Pool capacity
q_pooling.addConstrs((jk.sum(j,'*') <= cap[j] for j in pools),
                     name="Pool_capacity")
# 3. Target demand
q_pooling.addConstrs((ik.sum('*',k) + jk.sum('*',k) >= demand[k] for k in targets),
                     name="Target_demand")
# 4. Pool inflow
q_pooling.addConstrs((p_ij.sum('*',j) == 1 for j in pools),
                     name="Pool_inflow")
# 5.1 Target (min) tolerances
q_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(content[i][attr]*p_ij[i,j]*jk[j,k]
                                 for (i,j) in s2p if (j,k) in p2t)
                      >= min_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in min_tol[k].keys()),
                     name="Target_min_tolerances")
# 5.2 Target (max) tolerances
q_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(content[i][attr]*p_ij[i,j]*jk[j,k]
                                 for (i,j) in s2p if (j,k) in p2t)
                      <= max_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in max_tol[k].keys()),
                     name="Target_max_tolerances")

# Deploy Objective Function

# 0. Total profit
obj = gp.quicksum(price[k]*(ik.sum('*',k) + jk.sum('*',k))
               for k in targets) \
- gp.quicksum(cost[i]*(gp.quicksum(p_ij[i,j]*jk.sum(j,'*')
                             for j in pools if (i,j) in s2p)
                    + ik.sum(i,'*'))
           for i in sources)
q_pooling.setObjective(obj, GRB.MAXIMIZE)

# Find the optimal solution
q_pooling.optimize()




<p>The Q-formulation for this instance has:</p>
<ul>
<li>16 decision variables.</li>
<li>7 linear constraints.</li>
<li>15 bilinear constraints.</li>
<li>a bilinear objective function.</li>
</ul>
<p>Notice it has fewer decision variables and also fewer bilinear constraints. Now Gurobi was able to find the optimal solution of 439,182.59 USD in less than one second.</p>
<hr/>
<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Analysis">¶</a></h2><p>Let's see the optimal flows found:</p>
<h3 id="Flow-from-Sources-to-Targets">Flow from Sources to Targets<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Flow-from-Sources-to-Targets">¶</a></h3><p>The following table determines the flows from source nodes to target nodes. For example, from source node s2 to target node t1 there is a flow of 966.7<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;msup&gt;&lt;mn&gt;10&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-87-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1677" style="width: 2.682em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.205em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.134em, 1002.21em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1678"><span class="mo" id="MathJax-Span-1679" style="font-family: MathJax_Main;">×</span><span class="msubsup" id="MathJax-Span-1680"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.96em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-1681" style="font-family: MathJax_Main;">10</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 1.015em;"><span class="texatom" id="MathJax-Span-1682"><span class="mrow" id="MathJax-Span-1683"><span class="mn" id="MathJax-Span-1684" style="font-size: 70.7%; font-family: MathJax_Main;">2</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo>×</mo><msup><mn>10</mn><mrow class="MJX-TeXAtom-ORD"><mn>2</mn></mrow></msup></math></span></span><script id="MathJax-Element-87" type="math/tex">\times 10^{2}</script> bbl.</p>


In [ ]:

rows = sources.copy()
columns = targets.copy()
s2t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for source, target in ik.keys():
    if (abs(ik[source, target].x) > 1e-6):
        s2t_plan.loc[source, target] = np.round(ik[source, target].x, 1)
s2t_plan




<h3 id="Flow-from-Pools-to-Targets">Flow from Pools to Targets<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Flow-from-Pools-to-Targets">¶</a></h3><p>The following table defines the flows from pool nodes to target nodes. For example, from pool node p1 to target node t1 there is a flow of 92.8<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;msup&gt;&lt;mn&gt;10&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-88-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1685" style="width: 2.682em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.205em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.134em, 1002.21em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1686"><span class="mo" id="MathJax-Span-1687" style="font-family: MathJax_Main;">×</span><span class="msubsup" id="MathJax-Span-1688"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.96em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-1689" style="font-family: MathJax_Main;">10</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 1.015em;"><span class="texatom" id="MathJax-Span-1690"><span class="mrow" id="MathJax-Span-1691"><span class="mn" id="MathJax-Span-1692" style="font-size: 70.7%; font-family: MathJax_Main;">2</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo>×</mo><msup><mn>10</mn><mrow class="MJX-TeXAtom-ORD"><mn>2</mn></mrow></msup></math></span></span><script id="MathJax-Element-88" type="math/tex">\times 10^{2}</script> bbl.</p>


In [ ]:

rows = pools.copy()
columns = targets.copy()
p2t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for pool, target in jk.keys():
    if (abs(jk[pool, target].x) > 1e-6):
        p2t_plan.loc[pool, target] = np.round(jk[pool, target].x, 1)
p2t_plan




<h3 id="Flow-from-Sources-to-Pools">Flow from Sources to Pools<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Flow-from-Sources-to-Pools">¶</a></h3><p>The following table shows the flows from source nodes to pool nodes. For example, from source node s2 to pool node p1  there is a flow of 258.3<span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;msup&gt;&lt;mn&gt;10&lt;/mn&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;/mrow&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-89-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1693" style="width: 2.682em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.205em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.134em, 1002.21em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1694"><span class="mo" id="MathJax-Span-1695" style="font-family: MathJax_Main;">×</span><span class="msubsup" id="MathJax-Span-1696"><span style="display: inline-block; position: relative; width: 1.432em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.96em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mn" id="MathJax-Span-1697" style="font-family: MathJax_Main;">10</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 1.015em;"><span class="texatom" id="MathJax-Span-1698"><span class="mrow" id="MathJax-Span-1699"><span class="mn" id="MathJax-Span-1700" style="font-size: 70.7%; font-family: MathJax_Main;">2</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo>×</mo><msup><mn>10</mn><mrow class="MJX-TeXAtom-ORD"><mn>2</mn></mrow></msup></math></span></span><script id="MathJax-Element-89" type="math/tex">\times 10^{2}</script> bbl.</p>


In [ ]:

rows = sources.copy()
columns = pools.copy()
s2p_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for source, pool in p_ij.keys():
    if (abs(p_ij[source, pool].x) > 1e-6):
        flow = p_ij[source, pool].x * p2t_plan.loc[pool,:].sum()
        s2p_plan.loc[source, pool] = np.round(flow, 1)
s2p_plan




<hr/>
<h2 id="Conclusions">Conclusions<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#Conclusions">¶</a></h2><p>This notebook showed how easy it is to solve Bilinear Programs using Gurobi. It also highlighted the dramatic difference in performance of alternative formulations when solving challenging problems, such as the Standard Pooling Problem. It is thus of utmost importance to analyze carefully the context of the problem at hand, and to weigh the pros and cons of alternative models.</p>
<hr/>
<p><a id="references"></a></p>
<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/pooling/std_pooling.html#References">¶</a></h2><ol>
<li>Alfaki, M. (2012). Models and solution methods for the pooling problem.</li>
<li>Audet, C., Brimberg, J., Hansen, P., Digabel, S. L., &amp; Mladenović, N. (2004). Pooling problem: Alternate formulations and solution methods. Management science, 50(6), 761-776.</li>
<li>Dombrowski, J. (2015, June 07). McCormick envelopes. Retrieved from <a href="https://optimization.mccormick.northwestern.edu/index.php/McCormick_envelopes">https://optimization.mccormick.northwestern.edu/index.php/McCormick_envelopes</a></li>
<li>Gupte, A., Ahmed, S., Dey, S. S., &amp; Cheon, M. S. (2017). Relaxations and discretizations for the pooling problem. Journal of Global Optimization, 67(3), 631-669.</li>
<li>Haverly, C. A. (1978). Studies of the behavior of recursion for the pooling problem. Acm sigmap bulletin, (25), 19-28.</li>
<li>Liberti, L. (2008). Introduction to global optimization. Ecole Polytechnique.</li>
<li>Zhuang E. (2015, June 06). Spatial branch and bound method. Retrieved from
<a href="https://optimization.mccormick.northwestern.edu/index.php/Spatial_branch_and_bound_method">https://optimization.mccormick.northwestern.edu/index.php/Spatial_branch_and_bound_method</a></li>
</ol>
<p>Copyright © 2020 Gurobi Optimization, LLC</p>
